In [1]:
# !pip install langchain openai duckduckgo-search wikipedia beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup
from typing import Type
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from pydantic import BaseModel, Field
import os

# --- LLM 설정 ---
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4-1106-preview")

# --- Wikipedia 검색 도구 ---
class WikipediaSearchArgs(BaseModel):
    query: str = Field(description="Search term for Wikipedia")

class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"
    description: str = "Search for information on Wikipedia."
    args_schema: Type[WikipediaSearchArgs] = WikipediaSearchArgs

    def _run(self, query: str) -> str:
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)

    def _arun(self, query: str):
        raise NotImplementedError("Async not implemented")

# --- DuckDuckGo 검색 도구 ---
class DuckDuckGoSearchArgs(BaseModel):
    query: str = Field(description="Search query for DuckDuckGo")

class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGoSearchTool"
    description: str = "Search for web pages using DuckDuckGo."
    args_schema: Type[DuckDuckGoSearchArgs] = DuckDuckGoSearchArgs

    def _run(self, query: str) -> str:
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

    def _arun(self, query: str):
        raise NotImplementedError("Async not implemented")

# --- 웹 스크래핑 도구 ---
class WebScraperArgs(BaseModel):
    url: str = Field(description="The URL of the website to scrape")

class WebScraperTool(BaseTool):
    name: str = "WebScraperTool"
    description: str = "Fetch and extract main text content from a webpage"
    args_schema: Type[WebScraperArgs] = WebScraperArgs

    def _run(self, url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.text, "html.parser")
            paragraphs = [p.get_text() for p in soup.find_all("p")]
            text = "\n".join(paragraphs[:20])  # 앞부분 20개 문단만 추출
            return text if text else "No readable content found."
        except Exception as e:
            return f"Error scraping {url}: {str(e)}"

    def _arun(self, url: str):
        raise NotImplementedError("Async not implemented")

# --- 결과 저장 도구 ---
class SaveToFileArgs(BaseModel):
    content: str = Field(description="The research content to save")
    filename: str = Field(description="Filename to save the research", default="research_result.txt")

class SaveToFileTool(BaseTool):
    name: str = "SaveToFileTool"
    description: str = "Save research content into a .txt file"
    args_schema: Type[SaveToFileArgs] = SaveToFileArgs

    def _run(self, content: str, filename: str = "research_result.txt") -> str:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)
        return f"Research saved to {filename}"

    def _arun(self, content: str, filename: str = "research_result.txt"):
        raise NotImplementedError("Async not implemented")

# --- 에이전트 초기화 ---
tools = [
    WikipediaSearchTool(),
    DuckDuckGoSearchTool(),
    WebScraperTool(),
    SaveToFileTool(),
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True
)

# --- 실행 ---
query = "Research about the XZ backdoor"
agent.invoke(query)


C:\Users\kwon\AppData\Local\Temp\ipykernel_28156\1472154584.py:96: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`




c:\Users\kwon\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr--P7Goz9u-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and … Apr 1, 2024 · One of the core techniques used by the XZ backdoor to gain initial control during execution is the GNU Indirect Function (ifunc) … Apr 1, 2024 · CVE-2024-3094는 수많은 Linux 머신에 영향을 줄 수 있는 XZ Utils의 백도어입니다. 이에 대한 중요한 정보와 방어 단계를 공유합니다. Jun 4, 2024 · 본 문서에서는 리눅스 생태계의 보안에 영향을 미치고 있는 XZ Utils 백도어 사건을 유발하는 CVE-2024-3094의 공격방식과 백도어에 대한 분석을 통해 취약점의 영향도와 대응방안을 … XZ Utils 버전 5.6.0 (2월 24일 출시)과 5.6.1 (3월 9일 출시)에 무단 원격 접근을 허용하도록 설계된 악성코드를 추가해 백도어가 설치되어 배포되었습니다.The XZ backdoor refers to a malicious backdoor that was introduced into the Linux build of the xz utility within the liblzma library in versions 5.6.0 and later. This backdoor was identified by the CVE identifier CVE-2024-3094 and has the potential to affect numerous Linux machines.

One of the core techniques used by the XZ backdoor to gain initial control during execution is the GNU Indire

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor refers to a malicious backdoor that was introduced into the Linux build of the xz utility within the liblzma library in versions 5.6.0 and later. This backdoor was identified by the CVE identifier CVE-2024-3094 and has the potential to affect numerous Linux machines.\n\nOne of the core techniques used by the XZ backdoor to gain initial control during execution is the GNU Indirect Function (ifunc). This backdoor was included in the releases of XZ Utils version 5.6.0, which was released on February 24, and version 5.6.1, which was released on March 9. The malicious code was designed to allow unauthorized remote access.\n\nFor more detailed information and defensive measures regarding this backdoor, further research and analysis would be necessary to understand the impact of the vulnerability and the appropriate response strategies.'}